# Import

In [1]:
import numpy as np
import scipy.sparse as sp
import time
import torch
import scipy
import random
import pdb
import copy
import os
import argparse


import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import matplotlib.pyplot as plt

# from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv,GINConv,SAGEConv,GATConv,PNAConv, GraphSAGE
from torch_geometric.utils import to_dense_adj
from pygod.utils import load_data
from pygod.metrics import eval_roc_auc

/home/roy206/.conda/envs/cent7/2020.11-py38/NWRGAE/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Helper Function

In [2]:
def aug_random_edge(input_adj, perturb_percent=0.2, drop_edge = True, add_edge = True, self_loop = True):

    aug_adj = copy.deepcopy(input_adj)
    nb_nodes = input_adj.shape[0]
    edge_index = (input_adj>0).nonzero().t()
    
    edge_dict = {}
    for i in range(nb_nodes):
        edge_dict[i] = set()
    
    for edge in edge_index:
        i,j = edge[0],edge[1]
        i = i.item()
        j = j.item()
        edge_dict[i].add(j)
        edge_dict[j].add(i)
    
    if drop_edge: 
        for i in range(nb_nodes):
            d = len(edge_dict[i])
            node_list = list(edge_dict[i])
            num_edge_to_drop = int(d * perturb_percent)

            sampled_nodes = random.sample(node_list, num_edge_to_drop)

            for j in sampled_nodes:
                aug_adj[i][j] = 0
                aug_adj[j][i] = 0

            
    node_list = [i for i in range(nb_nodes)]
    # num_edge_to_add = int(nb_nodes * perturb_percent)
    
    add_list = []
    for i in range(nb_nodes):
        d = len(edge_dict[i])
        num_edge_to_add = int(d * perturb_percent)
        sampled_nodes = random.sample(node_list, num_edge_to_add)
        for j in sampled_nodes:
            add_list.append((i,j))
            
    if add_edge:
        for i in add_list:
            aug_adj[i[0]][i[1]] = 1
            aug_adj[i[1]][i[0]] = 1
    
    if self_loop: 
        for i in range(nb_nodes):
            aug_adj[i][i] = 1
            aug_adj[i][i] = 1
    
    
    return aug_adj


def _aug_random_edge(nb_nodes, edge_index, perturb_percent=0.2, drop_edge = True, add_edge = True, self_loop = True, use_avg_deg = True):

    
    total_edges = edge_index.shape[1]
    avg_degree = int(total_edges/nb_nodes)
    
    
    edge_dict = {}
    for i in range(nb_nodes):
        edge_dict[i] = set()
    
    for edge in edge_index:
        i,j = edge[0],edge[1]
        i = i.item()
        j = j.item()
        edge_dict[i].add(j)
        edge_dict[j].add(i)
    
    if drop_edge: 
        for i in range(nb_nodes):
            
            d = len(edge_dict[i])
            if use_avg_deg:
                num_edge_to_drop = avg_degree
            else:
                num_edge_to_drop = int(d * perturb_percent)

            node_list = list(edge_dict[i])
            num_edge_to_drop = min(num_edge_to_drop, d)
            sampled_nodes = random.sample(node_list, num_edge_to_drop)

            for j in sampled_nodes:
                edge_dict[i].discard(j)
                edge_dict[j].discard(i)
            
    node_list = [i for i in range(nb_nodes)]
    
    add_list = []
    for i in range(nb_nodes):
        
        if use_avg_deg:
            num_edge_to_add =  avg_degree
        else:
            d = len(edge_dict[i])
            num_edge_to_add = int(d * perturb_percent)
        
        sampled_nodes = random.sample(node_list, num_edge_to_add)
        for j in sampled_nodes:
            add_list.append((i,j))
            
    if add_edge:
        for edge in add_list:
            u = edge[0]
            v = edge[1]
            
            edge_dict[u].add(v)
            edge_dict[v].add(u)
    
    if self_loop: 
        for i in range(nb_nodes):
            edge_dict[i].add(i)
            
    updated_edges = set()
    for i in range(nb_nodes):
        for j in edge_dict[i]:
            updated_edges.add((i,j))
            updated_edges.add((j,i))
    
    row = []
    col = []
    for edge in updated_edges:
        u = edge[0]
        v = edge[1]
        row.append(u)
        col.append(v)
    
    aug_edge_index = [row,col]
    aug_edge_index = torch.tensor(aug_edge_index)
    
    return aug_edge_index


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    features = features.squeeze()
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features
    # return features, sparse_to_tuple(features)

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

# MLP

In [3]:
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        
        super(MLP, self).__init__()

        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = self.linears[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 0, 1)
                    h = torch.transpose(h, 1, 2)
                    
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 1, 2)
                    h = torch.transpose(h, 0, 1)

                h = F.relu(h)
                
            return self.linears[self.num_layers - 1](h)

# GNN Model

In [4]:
class GNN(nn.Module):
    def __init__(self, in_dim, out_dim, GNN_name = "GCN"):
    
        super(GNN, self).__init__()    
        
        self.mlp0 = MLP(3, in_dim, out_dim, out_dim)
        
        if GNN_name == "GIN":
            self.linear1 = MLP(4, out_dim, out_dim, out_dim)
            self.graphconv1 = GINConv(self.linear1)
        elif GNN_name == "GCN":
            self.graphconv1 = GCNConv(out_dim, out_dim, aggr='mean')
        elif GNN_name == "GAT":
            self.graphconv1 = GATConv(out_dim, out_dim, aggr='mean')
        elif GNN_name == "SAGE":
            self.graphconv1 = SAGEConv(out_dim, out_dim, aggr='mean')
            
        self.mlp1 = nn.Linear(out_dim,1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, x, edge_index):
        h0 = self.mlp0(x)
        h1 = self.graphconv1(h0,edge_index)
        h2 = self.mlp1(h1)
        h2 = self.relu(h2)
        p = torch.exp(h2)
        
        # h0 = self.mlp0(x)
        # h1 = self.mlp1(h0)
        # p = torch.exp(h1)
        return p
        
    

# Execution

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

parser = argparse.ArgumentParser("GAD-EBM: Graph Anomaly Detection via Energy-based Model")

parser.add_argument('-f')
parser.add_argument('--dataset',          type=str,           default="disney",         help='dataset name')
parser.add_argument('--perturb_percent',  type=float,         default=0.05,             help='perturb percent')
parser.add_argument('--seed',             type=int,           default=42,               help='seed')
parser.add_argument('--nb_epochs',        type=int,           default=200,              help='total epochs')
parser.add_argument('--hidden_dim',       type=int,           default=16,               help='hidden dimension')
parser.add_argument('--lr',               type=float,         default=0.01,             help='learning rate')
parser.add_argument('--l2_coef',          type=float,         default=0,                help='regularization coefficeint')
parser.add_argument('--gpu',              type=int,           default=0,                help='gpu')
parser.add_argument('--save_name',        type=str,           default='try.pkl',        help='save ckpt name')
parser.add_argument('--drop_edge',        type=bool,          default=True,             help='drop edge flag to produce state space neighbor')
parser.add_argument('--add_edge',         type=bool,          default=False,            help='add edge flag to produce state space neighbor')
parser.add_argument('--self_loop',        type=bool,          default=True,             help='self loop in state space neighbor')
parser.add_argument('--preprocess_feat',  type=bool,          default=True,             help='preprocess feature flag')
parser.add_argument('--use_avg_deg',      type=bool,          default=False,            help='use avg_deg to add/drop edge in state space neighbor')
parser.add_argument('--GNN_name',         type=str,           default="GCN",            help='gnn encoder')
parser.add_argument('--num_neigh',        type=int,           default=1,                help='state space graph number of neighbors')

args = parser.parse_args()

print('-' * 100)
print(args)
print('-' * 100)

dataset_str = args.dataset
perturb_percent = args.perturb_percent
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu) 
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

nb_epochs = args.nb_epochs
lr = args.lr
l2_coef = args.l2_coef
hidden_dim = args.hidden_dim
k = args.num_neigh



data = load_data(dataset_str)
edge_index = data.edge_index


adj = to_dense_adj(edge_index).squeeze()
features = data.x
labels = data.y
y = labels.bool()

anomaly_nodes = np.nonzero(y)

nb_nodes = features.shape[0]  # total node
input_dim = features.shape[1]   # total features



model = GNN(input_dim, hidden_dim, args.GNN_name)
optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)


if args.preprocess_feat:
    features = preprocess_features(features)

    
# if args.preprocess_adj:
#     adj = normalize_adj(adj)

    


st = time.time()
mx_auc = 0

aug_edge_indexes = []

for i in range(k):
    aug_edge_index = _aug_random_edge(nb_nodes,edge_index, perturb_percent=perturb_percent,drop_edge=args.drop_edge,
                                  add_edge=args.add_edge,self_loop=args.self_loop, use_avg_deg = args.use_avg_deg) # add/drop perturb percentage of edges
    
    
    aug_edge_index = aug_edge_index.to(device)
    
    aug_edge_indexes.append(aug_edge_index)
    
    
model = model.to(device)
features = torch.FloatTensor(features[np.newaxis])
features = features.to(device)
edge_index = edge_index.to(device)

losses = []
    
for epoch in range(nb_epochs):

    model.train()
    optimiser.zero_grad()
    
    p_data = model(features, edge_index)
    loss = 0
    
    for i in range(k):
        
        aug_edge_index = aug_edge_indexes[i]
        
        shuf_fts = features
        idx = np.random.permutation(nb_nodes)
        shuf_fts = features[:, idx, :]
        
        
        p_neigh = model(shuf_fts, aug_edge_index)
    
        c_theta_j1 = p_neigh/p_data
        c_theta_j2 = p_data/p_neigh
            
        j1 = (c_theta_j1**2 + 2 * c_theta_j1).mean()
        j2 = (2 * c_theta_j2).mean()
        
        
        
        neigh_loss = j1 - j2
        neigh_loss = neigh_loss.mean()
        loss = loss + neigh_loss
    
    loss = loss / k
    
    losses.append(loss)
    

    
    logits = p_data.squeeze().detach().cpu() 
    auc_score = eval_roc_auc(y.numpy(), logits.numpy()) * 100
    
    print("Loss: ", loss.item(), "AUC Score: ", auc_score)
    mx_auc = max(mx_auc, auc_score)
    
    loss.backward()
    optimiser.step()

en = time.time()


print("Maximum AUC: ", mx_auc)
print("Required Time: ", en-st)

----------------------------------------------------------------------------------------------------
Namespace(GNN_name='GCN', add_edge=False, dataset='disney', drop_edge=True, f='/home/roy206/.local/share/jupyter/runtime/kernel-fc3e89bb-cf5d-4676-bfed-54100c3c87fa.json', gpu=0, hidden_dim=16, l2_coef=0, lr=0.01, nb_epochs=200, num_neigh=1, perturb_percent=0.05, preprocess_feat=True, save_name='try.pkl', seed=42, self_loop=True, use_avg_deg=False)
----------------------------------------------------------------------------------------------------
2
Loss:  1.0129443407058716 AUC Score:  75.28248587570621
Loss:  0.9694161415100098 AUC Score:  35.3813559322034
Loss:  0.938948392868042 AUC Score:  31.85028248587571
Loss:  0.8653993606567383 AUC Score:  26.906779661016948
Loss:  0.7981433868408203 AUC Score:  26.412429378531066
Loss:  0.7775387763977051 AUC Score:  26.34180790960452
Loss:  0.7239422798156738 AUC Score:  26.271186440677962
Loss:  0.6635088920593262 AUC Score:  26.90677966101